In [ ]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
sommarioni_p = Path('../../1808_Sommarioni/')
sommarioni_geo_fp = list(sommarioni_p.rglob('sommarioni_geometries_202*.geojson'))[0]
sommarioni_txt_fp = list(Path('.').rglob('3_Sommarioni_with_new_transcriptions*.json'))[0]
gdf = gpd.read_file(sommarioni_geo_fp)
df = pd.read_json(sommarioni_txt_fp, lines = True)


# building a master dictionary of church names to Wikidata entry.

In [ ]:
import json
parish_df = gpd.read_file('../../1740_redrawn_parishes_cleaned_wikidata_standardised.geojson')
parish_df = parish_df[parish_df['church'].notnull()]
parish_df['wd'] = parish_df['church'].str.replace('http://www.wikidata.org/entity/', '')
parish_df

def set_dict_val_if_val(d: dict, key, val):
    if key not in d and key is not None:
        d[key] = val

name_wd_d = {}
for idx, row in parish_df.iterrows():
    wd = row['wd']
    set_dict_val_if_val(name_wd_d, row['NAME'], wd)
    set_dict_val_if_val(name_wd_d, row['alt_name'], wd)
    set_dict_val_if_val(name_wd_d, row['wd_venetian_name'], wd)
    set_dict_val_if_val(name_wd_d, row['wd_italian_name'], wd)

sparql_req_res = pd.read_csv('../Garzoni/all_wikidata_churches.csv').drop(columns = ['Unnamed: 0'])
sparql_req_res['wd'] = sparql_req_res['church'].str.replace('http://www.wikidata.org/entity/', '')
sparql_req_res = sparql_req_res[sparql_req_res['wd'].notnull()]
for idx, row in sparql_req_res.iterrows():
    set_dict_val_if_val(name_wd_d, row['italianLabel'], row['wd'])
    set_dict_val_if_val(name_wd_d, row['venetianLabel'], row['wd'])

with open('church_matching/name_wd_dict.json', 'w') as f:
    json.dump(name_wd_d, f, indent = 2)


# Adding the matches of church with wikidata.

In [ ]:
churches_matched = pd.read_csv('church_matching/special_churches_sommarioni_to_wd_id.csv')
unmatched = churches_matched[churches_matched['wd_id'].isnull()]
matched = churches_matched[churches_matched['wd_id'].notnull()]
matched
for _, row in matched.iterrows():
    #adding to owner standardized the wikidata id we matched.
    own_standardized = df.iloc[row['idx']]['owner_standardised']
    df.at[row['idx'], 'owner_standardised'] = f"{own_standardized}, ({row['wd_id']}"

df = df.replace('Sesta ', 'Scuola ')

In [ ]:
df.to_json('sommarioni_text_data_20250318_internal.json', orient='records', indent=2)